### Create data dictionary for w2v embeddings

In [ ]:
import pandas as pd

dir_to_parse = '/Volumes/Data/AHC IE DATA REQUESTS/PakhomovS-U01/'

In [ ]:
df =  pd.read_csv(dir_to_parse + 'test.txt',sep="|")
print(df.head())
print(df.columns)

In [ ]:
patient_id  = set(df['patient_id'].tolist())
print(patient_id)

In [ ]:
enc = pd.read_csv(dir_to_parse + 'service_dates.txt',sep="|")
enc['encounter_date'] = enc['encounter_date'].astype('datetime64[ns]')
print(enc.head())
print(enc.columns)

In [ ]:
import numpy as np

codes = pd.read_csv(dir_to_parse + 'test_icd9.txt',sep="|")
print(codes.head(), len(codes))
print(codes.columns)

# add random ints for testing
codes['icd9'] = np.random.randint(0,1000,size=(len(codes),1)) 
print(codes.head())

#print(codes['icd9'].copy().tolist())

In [ ]:
# Operations for random date shift: https://stackoverflow.com/questions/30926323/how-to-do-a-calculation-on-python-with-a-random-operator/30926409
operators = ["+","-"]  
import operator as op
oper = {'+': op.add, '-': op.sub}

In [ ]:
from datetime import timedelta
def shift_date(df, fn, n):

    df["date_shift"] = fn(df["encounter_date"], timedelta(days=n)).dt.strftime('%Y-%m-%d')
    
    return df

In [ ]:
import random
from random import randrange, uniform
import numpy as np

test = dict()

df['score'], df['concept'] = df['attributes'].str.split(',', 1).str
df['concept'] = df['concept'].str[15:-2] # handle extracting element from string
df = df.drop(['score'], axis=1)
df['neg'] = np.where(df['negated'] != 'F', '~', '')

print(df.head())
df['CUI'] = [{key: val} for key, val in zip(df.item, df.neg + df.concept)]

for i in patient_id:  
    print(i)
    service_ids = df['service_id'][df['patient_id']==i].copy()
   
    # merge encounter dates with notes
    dates = enc.merge(service_ids, left_on="service_id", right_on="service_id", how="inner").drop_duplicates()
    
    icd9 = codes.merge(service_ids, left_on="service_id", right_on="service_id", how="inner")
    
    # shift date by randrange: gives you an integral value
    irand = randrange(1, 365)
    #picked_operator = random.choice(oper)
    # get random oepration
    fn = oper[random.choice(operators)]
    
    #print(irand, picked_operator)
   
    # iterate over encounters
    for s in set(service_ids.tolist()):
        
        enc_date = dates[dates['service_id'] == s].copy()
        icdcodes = codes[codes['service_id'] == s].copy()
       
        # deidentified patient_id
        enc_date['uuid'] = enc_date['uuid'].str.replace('}','')
        enc_date['uuid'] = enc_date['uuid'].str.replace('{','')
        uuid = enc_date['uuid'].values[0]
        #print('ttest', uuid)
        
        #print(shift_date(enc_date, picked_operator, irand))
        
        # shift date by irand and rand operator
        rand_date = shift_date(enc_date, fn, irand)['date_shift'].values[0]
        #print('test', rand_date)
        
        df1 = df[df['service_id'] == s].copy()
    
        # get series of CUIS
        cuis = df1['CUI'].copy()
        icd = icdcodes['icd9'].copy()
        
        icd = icd.tolist()
        
        # get count for CUIs 
        df2 = df1.groupby('item').first().copy()
        df2['n'] = df1['item'].value_counts()
        df2.reset_index(inplace=True)
        counts = df2[['item','n']].copy()
        counts['count'] = [{key: val} for key, val in zip(counts.item, counts.n)]
        
        # encounter key
        #s = str(s)+str(rand_date)
        s = str(uuid)+str(rand_date)
        
        
#         if str(i) == str(3822760233):
        if uuid not in test: # if i not in test:
            # list of dictionaries of cuis, iem counts, respectitvely, to list of single dictionary
            a = {k: v for d in cuis.tolist() for k, v in d.items()}
            b = {k: v for d in counts['count'].tolist() for k, v in d.items()}
            c = {'icd9': icd}
            
            #test[i] = {s: [[a, b], rand_date]}
            test[uuid] = {s: [c, [a, b], rand_date]}
        else:
            a = {k: v for d in cuis.tolist() for k, v in d.items()}
            b = {k: v for d in counts['count'].tolist() for k, v in d.items()}
            c = {'icd9': icd}
            #test[i].update({s: [[a, b], rand_date]})
            test[uuid].update({s: [c, [a, b], rand_date]})
        
        
# for k, v in dd[3822760232].items():
#     print(k)
    
import json
with open('file_icd.txt', 'w') as file:
     file.write(json.dumps(test))

In [ ]:
#service_ids

#cuis = df[['item', 'attributes']][df['service_id'] == 20322501260].copy()

df['score'], df['concept'] = df['attributes'].str.split(',', 1).str
df1 = df.copy()
df1['concept'] = df1['concept'].str[15:-1].copy()
df1 = df1.drop(['score'], axis=1)
df1['Merged'] = [{key: val} for key, val in zip(df1.item, df1.concept)]
print(df1.head())
